## Kaggle is slow - you'll have to wait **5 minutes** for it to install.

In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = HF_TOKEN, # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Data Preparation

In [5]:
prompt = """Below is an instruction that describes a task, paired with an optional input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["Prompt"]
    # Create an empty input field
    inputs = [""] * len(instructions)
    outputs = examples["Response"]
    texts = []
    for instruction, inp, output in zip(instructions, inputs, outputs):
        # Format the text using the prompt template and append the EOS token.
        text = prompt.format(instruction, inp, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

In [6]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="/kaggle/input/sleep-and-fitness-dataset/PH-LLM Custom Dataset.csv")["train"]

# Map the formatting function onto the dataset.
dataset = dataset.map(formatting_prompts_func, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1730 [00:00<?, ? examples/s]

In [7]:
dataset

Dataset({
    features: ['Category', 'ID', 'Prompt', 'Response', 'text'],
    num_rows: 1730
})

In [8]:
print(f'Prompt: \n{dataset[0]["Prompt"]}')
print(f'Response: \n{dataset[0]["Response"]}')

Prompt: 
You are a sleep medicine expert. You are given the following sleep data.
The user is Female, 65 years old.
Sleep Summary: 
Bedtime: 2021-03-06 01:00:00
Wakeup time: 2021-03-06 07:00:00
Sleep duration: 6.0
Sleep efficiency: 0.88
REM sleep percentage: 18
Deep sleep percentage: 70
Light sleep percentage: 12
Awakenings: 0.0
Caffeine consumption: 0.0
Alcohol consumption: 0.0
Smoking status: Yes
Exercise frequency: 3.0

List the most important insights. Identify all of the patterns of data that are likely out of the preferred range. Make sure to consider various sleep health dimensions: Routine, Sleep Quality, Alertness, Timing, Efficiency, and Duration. Add a heading for each dimension. Optionally (only do this if extremely important) add a heading called Other for anything else that doesn't fit the above categories. For Routine, consider the average bedtime, wake time, midsleep point and standard deviations of these, focus on the consistency of the routine, not timing. For Sleep Q

# Model Training

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 60,

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/1730 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/1730 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1730 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1730 [00:00<?, ? examples/s]

In [10]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.516 GB of memory reserved.


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,730 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.898400
2,1.938100
3,1.714500
4,1.804400
5,1.695100
6,1.554900
7,1.462200
8,1.517300
9,1.476400
10,1.277900


In [12]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1714.9843 seconds used for training.
28.58 minutes used for training.
Peak reserved memory = 7.428 GB.
Peak reserved memory for training = 1.912 GB.
Peak reserved memory % of max memory = 50.39 %.
Peak reserved memory for training % of max memory = 12.971 %.


### Inference

You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [20]:
# prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        """You are a sleep medicine expert. You are given the following sleep data.
        The user is Male, 55 years old.
        Sleep Summary:
        Bedtime: 2021-06-20 00:30:00
        Wakeup time: 2021-06-20 08:00:00
        Sleep duration: 7.5
        Sleep efficiency: 0.90
        REM sleep percentage: 20
        Deep sleep percentage: 50
        Light sleep percentage: 30
        Awakenings: 1.5
        Caffeine consumption: 1.0
        Alcohol consumption: 0.0
        Smoking status: Yes
        Exercise frequency: 3.5
        
        List the most important insights. Identify all patterns that deviate from the preferred range. Consider sleep health dimensions including Routine, Sleep Quality, Alertness, Timing, Efficiency, and Duration. Provide headings for each, and optionally include an Other category if necessary. Focus on consistency of routine, quality of sleep phases, and efficiency compared to standard percentiles. Avoid generic language and ensure clarity and conciseness.
        """ , # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                   streamer = text_streamer, max_new_tokens = 512, pad_token_id = tokenizer.eos_token_id)

Here's an analysis of your sleep data:

**Routine**

*   **Bedtime:** 12:30 AM is a late bedtime. Consistently going to bed this late may negatively impact your circadian rhythm, particularly if it's not aligned with your natural sleep-wake cycle.
*   **Wakeup Time:** 8:00 AM is a reasonable wake-up time, but if it's consistently later than your typical wake-up time, it could disrupt your sleep schedule.

**Sleep Quality**

*   **Deep Sleep Percentage:** 50% is significantly above the normal range (15-25%). While deep sleep is beneficial, such a high percentage may indicate an imbalance in sleep stages. It's worth noting that the data may not be entirely accurate.
*   **Light Sleep Percentage:** 30% is within the normal range (40-55%).
*   **REM Sleep Percentage:** 20% is within the normal range (20-25%).
*   **Awakenings:** 1.5 awakenings per night is slightly above the ideal (0-1), which could be related to smoking or caffeine.

**Alertness**

*   **No naps were reported.**

**Timing

### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [21]:
model.save_pretrained("PH-LLM-Llama-3.1-8B-Instruct-bnb-4bit")
tokenizer.save_pretrained("PH-LLM-Llama-3.1-8B-Instruct-bnb-4bit")

('PH-LLM-Llama-3.1-8B-Instruct-bnb-4bit/tokenizer_config.json',
 'PH-LLM-Llama-3.1-8B-Instruct-bnb-4bit/special_tokens_map.json',
 'PH-LLM-Llama-3.1-8B-Instruct-bnb-4bit/tokenizer.json')

In [22]:
model.push_to_hub("johnjehiel/PH-LLM-Llama-3.1-8B-Instruct-bnb-4bit", token=HF_TOKEN)
tokenizer.push_to_hub("johnjehiel/PH-LLM-Llama-3.1-8B-Instruct-bnb-4bit", token=HF_TOKEN)

README.md:   0%|          | 0.00/609 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/johnjehiel/PH-LLM-Llama-3.1-8B-Instruct-bnb-4bit


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]